# Food Vader

In [34]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

import requests as req
import pandas as pd
import numpy as np
import tweepy
import time
import json
import plotly.plotly as py
import plotly
from plotly.graph_objs import *

# import config file to keep keys secret
import config as c

# Define API Keys

In [7]:
# yelp info
API_KEY= c.yKey

# Zomato key
zKey = c.zKey

# Define Yelp! Functions

In [77]:
def request(host, path, api_key, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    print(u'Querying {0} ...'.format(url))
    response = req.request('GET', url, headers=headers, params=url_params)
    return response.json()

def search(api_key, term, location):
    OFFSET = 0
    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'sort_by': 'rating',
        'limit':SEARCH_LIMIT,
        'offset': OFFSET,
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
SEARCH_LIMIT = 50
OFFSET = 0

In [73]:
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except:
    pass

# Zomato API Call & DataFrame Building

In [74]:
# prompt user for input city, generate url for response, get city ID to use in loop
city_lookup = 'Austin'
url = "https://developers.zomato.com/api/v2.1/cities?&q=%s&results=100" % (city_lookup)
response = req.get(url, headers={"user-key" : zKey}).json()
city_id = response['location_suggestions'][0]['id']
city_name = response['location_suggestions'][0]['name']

# build empty lists to hold restaurant info
zomato_dict = {
    "Name":[],
    "Zomato Rating":[],
    "Zomato Review Count":[],
    "Address":[],
    "Zip Code":[],
    "Longitude":[],
    "Latitude":[],
    "Cuisine Type":[],
    "City Name":[]
}

# start loop to request restaurant info
start = 0
for x in range(5):
    url = 'https://developers.zomato.com/api/v2.1/search?entity_id=%s&entity_type=city&sort=rating&order=desc&start=%s&count=500' % (city_id, start)
    response = req.get(url, headers={'user-key':zKey}).json()
    for x in range(len(response['restaurants'])):
        zomato_dict['Name'].append(response['restaurants'][x]['restaurant']['name'])
        zomato_dict['Zomato Rating'].append(response['restaurants'][x]['restaurant']['user_rating']['aggregate_rating'])
        zomato_dict['Zomato Review Count'].append(response['restaurants'][x]['restaurant']['user_rating']['votes'])
        zomato_dict['Address'].append(response['restaurants'][x]['restaurant']['location']['address'])
        zomato_dict['Zip Code'].append(response['restaurants'][x]['restaurant']['location']['zipcode'])
        zomato_dict['Longitude'].append(response['restaurants'][x]['restaurant']['location']['longitude'])
        zomato_dict['Latitude'].append(response['restaurants'][x]['restaurant']['location']['latitude'])
        zomato_dict['Cuisine Type'].append(response['restaurants'][x]['restaurant']['cuisines'])
        zomato_dict['City Name'].append(city_name)
    start = start + 20

In [75]:
zomato_df = pd.DataFrame(zomato_dict)
zomato_df.head(10)

,Address,City Name,Cuisine Type,Latitude,Longitude,Name,Zip Code,Zomato Rating,Zomato Review Count
0,"1415 S Congress Avenue, Austin 78704","Austin, TX",Pizza,30.2493111111,-97.7496722222,Home Slice Pizza,78704,4.9,1004
1,"7720 Highway 71 W, Austin 78735","Austin, TX","Burger, Southwestern",30.2437694444,-97.8827666667,Jack Allen's Kitchen,78735,4.8,392
2,"303 Red River Street, Austin 78701","Austin, TX",Southern,30.2641944444,-97.7379444444,Moonshine Patio Bar & Grill,78701,4.7,905
3,"1400 S Congress Avenue, Austin 78704","Austin, TX",Burger,30.2497657000,-97.7501222000,Hopdoddy Burger Bar,78704,4.7,676
4,"900 East 11th Street, Austin 78702","Austin, TX",BBQ,30.2700861111,-97.7314611111,Franklin Barbecue,78702,4.6,551
5,"1503 S 1st Street, Austin 78704","Austin, TX","Donuts, Bakery, American",30.2494583333,-97.7550722222,Gourdough's,78704,4.6,497
6,"5501 N Lamar Boulevard, Suite C101, Austin 78751","Austin, TX",Thai,30.3229490000,-97.7278440000,Titaya's Thai Cuisine,78751,4.6,266
7,"1600 S Congress Avenue, Austin 78704","Austin, TX","Cafe, American, Bar Food",30.2478416667,-97.7504972222,South Congress Café,78704,4.5,603
8,"801 S Lamar Boulevard, Austin 78704","Austin, TX","Sushi, Japanese",30.2577277778,-97.7598277778,Uchi,78704,4.5,725
9,"1920 S Congress Avenue, Austin 78704","Austin, TX","American, Cafe",30.2444888889,-97.7517861111,Magnolia Cafe,78704,4.5,769


# Yelp API Call & DataFrame Building

In [76]:
yelp_response = search(API_KEY, 'restaurant', 'Austin')
yelp_response

Querying https://api.yelp.com/v3/businesses/search ...


{'error': {'code': 'TOO_LARGE_VALIDATION_ERROR',
  'description': 'Number too big: 100'}}

In [70]:
yelp_dict = {
    "Name":[],
    "Yelp Rating":[],
    "Yelp Review Count":[],
    "Address":[],
    "Zip Code":[],
    "Longitude":[],
    "Latitude":[],
    "Cuisine Type":[],
    "City Name":[]
}

for x in range(len(yelp_response['businesses'])):
    yelp_dict['Name'].append(yelp_response['businesses'][x]['name'])
    yelp_dict['Yelp Rating'].append(yelp_response['businesses'][x]['rating'])
    yelp_dict['Yelp Review Count'].append(yelp_response['businesses'][x]['review_count'])
    yelp_dict['Address'].append(yelp_response['businesses'][x]['location']['address1'])
    yelp_dict['Zip Code'].append(yelp_response['businesses'][x]['location']['zip_code'])
    yelp_dict['Longitude'].append(yelp_response['businesses'][x]['coordinates']['longitude'])
    yelp_dict['Latitude'].append(yelp_response['businesses'][x]['coordinates']['latitude'])
    yelp_dict['Cuisine Type'].append(yelp_response['businesses'][x]['categories'][0]['alias'])
    yelp_dict['City Name'].append(yelp_response['businesses'][x]['location']['city'])

In [79]:
yelp_df = pd.DataFrame(yelp_dict)
yelp_df.head()

,Address,City Name,Cuisine Type,Latitude,Longitude,Name,Yelp Rating,Yelp Review Count,Zip Code
0,1403 E 7th St,Austin,foodtrucks,30.264310,-97.727890,Boteco,5.0,330,78702
1,5000 Burnet Rd,Austin,hotdog,30.321169,-97.739506,T-Loc's Sonora Hot Dogs,5.0,313,78756
2,2500 E 6th St,Austin,thai,30.259010,-97.714450,DEE DEE,5.0,265,78702
3,900 E 11th St,Austin,bbq,30.270090,-97.731260,Franklin Barbecue,4.5,3835,78702
4,11800 N Lamar Blvd,Austin,greek,30.387600,-97.683990,Santorini Cafe,4.5,554,78753


In [81]:
# offset yelp API to search for next 50 restaurants

def search(api_key, term, location):
    OFFSET = 0
    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'sort_by': 'rating',
        'limit':SEARCH_LIMIT,
        'offset': 50,
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

In [83]:
yelp_response = search(API_KEY, 'restaurant', 'Austin')

Querying https://api.yelp.com/v3/businesses/search ...


In [91]:
yelp_dict2 = {
    "Name":[],
    "Yelp Rating":[],
    "Yelp Review Count":[],
    "Address":[],
    "Zip Code":[],
    "Longitude":[],
    "Latitude":[],
    "Cuisine Type":[],
    "City Name":[]
}

for x in range(len(yelp_response['businesses'])):
    yelp_dict2['Name'].append(yelp_response['businesses'][x]['name'])
    yelp_dict2['Yelp Rating'].append(yelp_response['businesses'][x]['rating'])
    yelp_dict2['Yelp Review Count'].append(yelp_response['businesses'][x]['review_count'])
    yelp_dict2['Address'].append(yelp_response['businesses'][x]['location']['address1'])
    yelp_dict2['Zip Code'].append(yelp_response['businesses'][x]['location']['zip_code'])
    yelp_dict2['Longitude'].append(yelp_response['businesses'][x]['coordinates']['longitude'])
    yelp_dict2['Latitude'].append(yelp_response['businesses'][x]['coordinates']['latitude'])
    yelp_dict2['Cuisine Type'].append(yelp_response['businesses'][x]['categories'][0]['alias'])
    yelp_dict2['City Name'].append(yelp_response['businesses'][x]['location']['city'])

In [92]:
# convert second response to new dataframe, append to original dataframe
yelp_df2 = pd.DataFrame(yelp_dict2)
yelp_df = yelp_df.append(yelp_df2, ignore_index=True)

In [94]:
yelp_df.head(100)

,Address,City Name,Cuisine Type,Latitude,Longitude,Name,Yelp Rating,Yelp Review Count,Zip Code
0,1403 E 7th St,Austin,foodtrucks,30.264310,-97.727890,Boteco,5.0,330,78702
1,5000 Burnet Rd,Austin,hotdog,30.321169,-97.739506,T-Loc's Sonora Hot Dogs,5.0,313,78756
2,2500 E 6th St,Austin,thai,30.259010,-97.714450,DEE DEE,5.0,265,78702
3,900 E 11th St,Austin,bbq,30.270090,-97.731260,Franklin Barbecue,4.5,3835,78702
4,11800 N Lamar Blvd,Austin,greek,30.387600,-97.683990,Santorini Cafe,4.5,554,78753
5,502 W 30th St,Austin,peruvian,30.296340,-97.741390,Ceviche7,5.0,197,78705
6,3595 Ranch Rd 620 S,Bee Cave,foodtrucks,30.320165,-97.952498,BackDraft Pizzeria,5.0,218,78738
7,2101 Manor Rd,Austin,mediterranean,30.283955,-97.719837,Abo Youssef,4.5,311,78722
8,8909 Burnet Rd,Austin,burgers,30.371860,-97.725860,Lebowski's Grill,4.5,278,78757
9,1100 Grand Ave Pkwy,Pflugerville,ethiopian,30.458330,-97.650940,Taste of Ethiopia,4.5,619,78660


In [96]:
zomato_df.head(100)

,Address,City Name,Cuisine Type,Latitude,Longitude,Name,Zip Code,Zomato Rating,Zomato Review Count
0,"1415 S Congress Avenue, Austin 78704","Austin, TX",Pizza,30.2493111111,-97.7496722222,Home Slice Pizza,78704,4.9,1004
1,"7720 Highway 71 W, Austin 78735","Austin, TX","Burger, Southwestern",30.2437694444,-97.8827666667,Jack Allen's Kitchen,78735,4.8,392
2,"303 Red River Street, Austin 78701","Austin, TX",Southern,30.2641944444,-97.7379444444,Moonshine Patio Bar & Grill,78701,4.7,905
3,"1400 S Congress Avenue, Austin 78704","Austin, TX",Burger,30.2497657000,-97.7501222000,Hopdoddy Burger Bar,78704,4.7,676
4,"900 East 11th Street, Austin 78702","Austin, TX",BBQ,30.2700861111,-97.7314611111,Franklin Barbecue,78702,4.6,551
5,"1503 S 1st Street, Austin 78704","Austin, TX","Donuts, Bakery, American",30.2494583333,-97.7550722222,Gourdough's,78704,4.6,497
6,"5501 N Lamar Boulevard, Suite C101, Austin 78751","Austin, TX",Thai,30.3229490000,-97.7278440000,Titaya's Thai Cuisine,78751,4.6,266
7,"1600 S Congress Avenue, Austin 78704","Austin, TX","Cafe, American, Bar Food",30.2478416667,-97.7504972222,South Congress Café,78704,4.5,603
8,"801 S Lamar Boulevard, Austin 78704","Austin, TX","Sushi, Japanese",30.2577277778,-97.7598277778,Uchi,78704,4.5,725
9,"1920 S Congress Avenue, Austin 78704","Austin, TX","American, Cafe",30.2444888889,-97.7517861111,Magnolia Cafe,78704,4.5,769
